In [ ]:
import cv2
import numpy as np
from collections import deque

def non_max_suppression(boxes, scores, threshold):
    """Apply Non-Maximum Suppression"""
    if len(boxes) == 0:
        return []
    
    boxes = np.array(boxes, dtype=np.float32)
    pick = []
    
    x1 = boxes[:, 0]
    y1 = boxes[:, 1]
    x2 = boxes[:, 0] + boxes[:, 2]
    y2 = boxes[:, 1] + boxes[:, 3]
    
    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    idxs = np.argsort(scores)
    
    while len(idxs) > 0:
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)
        
        xx1 = np.maximum(x1[i], x1[idxs[:last]])
        yy1 = np.maximum(y1[i], y1[idxs[:last]])
        xx2 = np.minimum(x2[i], x2[idxs[:last]])
        yy2 = np.minimum(y2[i], y2[idxs[:last]])
        
        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)
        
        overlap = (w * h) / area[idxs[:last]]
        
        idxs = np.delete(idxs, np.concatenate(([last],
            np.where(overlap > threshold)[0])))
    
    return pick

class VideoProcessor:
    def __init__(self):
        self.mog = cv2.createBackgroundSubtractorMOG2(
            history=1000,
            varThreshold=16,
            detectShadows=True
        )
        
        self.min_contour_area = 200
        self.max_trace_length = 50
        self.tracks = {}
        self.track_colors = {}
        self.next_track_id = 0
        self.max_tracking_distance = 70
        
        # NMS parameters
        self.nms_threshold = 0.3
        self.detection_confidence_threshold = 0.5
        
        self.frames_until_cleanup = 10
        self.track_activity = {}
        self.frame_dimensions = None
        self.margin = 20
        
        self.lower_bound = np.array([0, 0, 0])
        self.upper_bound = np.array([180, 255, 255])
        
        self.min_area = 10000
        self.dilate_kernel = np.ones((5,5), np.uint8)
        self.road_mask = None

        self.temporal_buffer = deque(maxlen=5) 

        
        cv2.namedWindow('Road Mask Controls')
        cv2.createTrackbar('Hue Min', 'Road Mask Controls', 103, 180, self.nothing)
        cv2.createTrackbar('Hue Max', 'Road Mask Controls', 180, 180, self.nothing)
        cv2.createTrackbar('Sat Min', 'Road Mask Controls', 0, 255, self.nothing)
        cv2.createTrackbar('Sat Max', 'Road Mask Controls', 255, 255, self.nothing)
        cv2.createTrackbar('Val Min', 'Road Mask Controls', 0, 255, self.nothing)
        cv2.createTrackbar('Val Max', 'Road Mask Controls', 255, 255, self.nothing)
        cv2.createTrackbar('Min Area', 'Road Mask Controls', 10000, 20000, self.nothing)
        cv2.createTrackbar('Dilate', 'Road Mask Controls', 20, 20, self.nothing)
        cv2.createTrackbar('Min Vehicle Size', 'Road Mask Controls', 200, 1000, self.nothing)
        cv2.createTrackbar('Detection Sensitivity', 'Road Mask Controls', 12, 50, self.nothing)
        cv2.createTrackbar('NMS Threshold', 'Road Mask Controls', 30, 100, self.nothing)
    
    def apply_temporal_filtering(self, mask):
            """
            Smooth the mask over time using the temporal buffer.
            """
            self.temporal_buffer.append(mask)
            combined_mask = np.sum(self.temporal_buffer, axis=0) / len(self.temporal_buffer)
            return (combined_mask > 128).astype(np.uint8) * 255  # Threshold to binarize the mask
    
    def apply_enhanced_mask(self, frame, road_mask):
        """
        Apply enhanced masking techniques to better isolate vehicles
        """
        # Get foreground mask from MOG2
        fg_mask = self.mog.apply(frame)
        
        # Convert frame to grayscale for additional processing
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # Apply adaptive thresholding to handle varying lighting
        thresh = cv2.adaptiveThreshold(
            gray, 255,
            cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
            cv2.THRESH_BINARY_INV, 11, 2
        )
        
        # Edge detection to highlight vehicle boundaries
        edges = cv2.Canny(gray, 50, 150)
        
        # Combine edges with threshold
        combined = cv2.bitwise_or(thresh, edges)
        
        # Create motion mask
        kernel = np.ones((3,3), np.uint8)
        motion_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_OPEN, kernel)
        motion_mask = cv2.morphologyEx(motion_mask, cv2.MORPH_CLOSE, kernel)
        
        # Combine motion with edges/threshold
        enhanced_mask = cv2.bitwise_and(combined, motion_mask)
        
        # Apply road mask
        final_mask = cv2.bitwise_and(enhanced_mask, enhanced_mask, mask=road_mask)
        
        # Clean up noise
        final_mask = cv2.medianBlur(final_mask, 3)
        final_mask = cv2.morphologyEx(final_mask, cv2.MORPH_CLOSE, 
                                    np.ones((5,5), np.uint8))
        
        return final_mask

    def nothing(self, x):
        pass

    def get_random_color(self):
        return (np.random.randint(0, 255),
                np.random.randint(0, 255),
                np.random.randint(0, 255))

    def is_point_in_frame(self, point):
        if self.frame_dimensions is None:
            return True
        
        x, y = point
        width, height = self.frame_dimensions
        return (-self.margin <= x <= width + self.margin and 
                -self.margin <= y <= height + self.margin)

    def is_point_on_road(self, point, road_mask):
        x, y = point
        if not self.is_point_in_frame((x, y)):
            return False
        
        if y >= road_mask.shape[0] or x >= road_mask.shape[1] or x < 0 or y < 0:
            return False
            
        return road_mask[y, x] > 0

    def cleanup_tracks(self, current_frame, road_mask):
        active_tracks = {}
        active_colors = {}
        active_activity = {}
        
        for track_id, track in self.tracks.items():
            if not track:
                continue
                
            last_point = track[-1]
            
            if (current_frame - self.track_activity.get(track_id, 0) <= self.frames_until_cleanup and
                self.is_point_in_frame(last_point) and
                self.is_point_on_road(last_point, road_mask)):
                active_tracks[track_id] = track
                active_colors[track_id] = self.track_colors[track_id]
                active_activity[track_id] = self.track_activity[track_id]
        
        self.tracks = active_tracks
        self.track_colors = active_colors
        self.track_activity = active_activity

    def update_mask_parameters(self):
        h_min = cv2.getTrackbarPos('Hue Min', 'Road Mask Controls')
        h_max = cv2.getTrackbarPos('Hue Max', 'Road Mask Controls')
        s_min = cv2.getTrackbarPos('Sat Min', 'Road Mask Controls')
        s_max = cv2.getTrackbarPos('Sat Max', 'Road Mask Controls')
        v_min = cv2.getTrackbarPos('Val Min', 'Road Mask Controls')
        v_max = cv2.getTrackbarPos('Val Max', 'Road Mask Controls')
        
        self.lower_bound = np.array([h_min, s_min, v_min])
        self.upper_bound = np.array([h_max, s_max, v_max])
        
        self.min_area = cv2.getTrackbarPos('Min Area', 'Road Mask Controls')
        dilate_size = cv2.getTrackbarPos('Dilate', 'Road Mask Controls')
        self.dilate_kernel = np.ones((dilate_size, dilate_size), np.uint8)
        
        self.min_contour_area = cv2.getTrackbarPos('Min Vehicle Size', 'Road Mask Controls')
        sensitivity = cv2.getTrackbarPos('Detection Sensitivity', 'Road Mask Controls')
        self.mog.setVarThreshold(sensitivity)

    def get_road_mask(self, frame):
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        mask = cv2.inRange(hsv, self.lower_bound, self.upper_bound)
        
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, np.ones((5,5),np.uint8))
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, np.ones((5,5),np.uint8))
        
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        filtered_mask = np.zeros_like(mask)
        
        for contour in contours:
            if cv2.contourArea(contour) > self.min_area:
                cv2.drawContours(filtered_mask, [contour], -1, 255, -1)
        
        filtered_mask = cv2.dilate(filtered_mask, self.dilate_kernel, iterations=1)
        return filtered_mask

    def find_closest_track(self, point):
        min_dist = float('inf')
        closest_id = None
        
        for track_id, track in self.tracks.items():
            if track:
                last_point = track[-1]
                dist = np.sqrt((point[0] - last_point[0])**2 + 
                             (point[1] - last_point[1])**2)
                if dist < min_dist and dist < self.max_tracking_distance:
                    min_dist = dist
                    closest_id = track_id
        return closest_id

    def process_detections(self, contours):
        boxes = []
        scores = []
        centroids = []
        
        for contour in contours:
            area = cv2.contourArea(contour)
            if area > self.min_contour_area:
                x, y, w, h = cv2.boundingRect(contour)
                aspect_ratio = float(w)/h
                
                if 0.4 < aspect_ratio < 2.5:
                    area_score = min(area / 2000.0, 1.0)
                    aspect_score = 1.0 - abs(1.5 - aspect_ratio) / 1.5
                    confidence = (area_score + aspect_score) / 2.0
                    
                    if confidence > self.detection_confidence_threshold:
                        boxes.append([x, y, w, h])
                        scores.append(confidence)
                        centroids.append((int(x + w/2), int(y + h/2)))
        
        if boxes:
            nms_threshold = cv2.getTrackbarPos('NMS Threshold', 'Road Mask Controls') / 100.0
            keep_indices = non_max_suppression(boxes, scores, nms_threshold)
            
            filtered_boxes = [boxes[i] for i in keep_indices]
            filtered_scores = [scores[i] for i in keep_indices]
            filtered_centroids = [centroids[i] for i in keep_indices]
            
            return filtered_boxes, filtered_scores, filtered_centroids
        
        return [], [], []

   
    def process_frame(self, frame, frame_count):
            if self.frame_dimensions is None:
                self.frame_dimensions = (frame.shape[1], frame.shape[0])
                
            self.update_mask_parameters()
            road_mask = self.get_road_mask(frame)
            road_mask = self.apply_temporal_filtering(road_mask)
            
            self.cleanup_tracks(frame_count, road_mask)
            
            fg_mask = self.apply_enhanced_mask(frame, road_mask)
            
            contours, _ = cv2.findContours(fg_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            
            boxes, scores, centroids = self.process_detections(contours)
            
            for box, score, centroid in zip(boxes, scores, centroids):
                if self.is_point_on_road(centroid, road_mask):
                    track_id = self.find_closest_track(centroid)
                    
                    if track_id is None:
                        track_id = self.next_track_id
                        self.next_track_id += 1
                        self.tracks[track_id] = deque(maxlen=self.max_trace_length)
                        self.track_colors[track_id] = self.get_random_color()
                    
                    self.tracks[track_id].append(centroid)
                    self.track_activity[track_id] = frame_count
                    
                    # Draw bounding box and confidence score
                    x, y, w, h = box
                    color = self.track_colors[track_id]
                    cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
                    
                    # Draw vehicle ID and confidence score
                    text = f'ID:{track_id} ({score:.2f})'
                    text_size = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 2)[0]
                    
                    # Draw background rectangle for text
                    cv2.rectangle(frame, 
                                (x, y - text_size[1] - 10), 
                                (x + text_size[0], y), 
                                color, 
                                -1)
                    
                    # Draw text in white for better visibility
                    cv2.putText(frame, text, (x, y-5), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
            
            # Draw tracking trails
            for track_id, track in self.tracks.items():
                points = list(track)
                if len(points) >= 2:
                    color = self.track_colors[track_id]
                    # Draw trail lines with fading effect
                    for i in range(1, len(points)):
                        alpha = float(i) / len(points)
                        thickness = int(alpha * 3) + 1
                        cv2.line(frame, points[i-1], points[i],
                                tuple(int(c * alpha) for c in color), thickness)
                    
                    # Draw current position marker
                    cv2.circle(frame, points[-1], 6, color, -1)
                    cv2.circle(frame, points[-1], 8, color, 2)
            
            road_overlay = frame.copy()
            road_overlay[road_mask == 0] = [0, 0, 0]
            frame = cv2.addWeighted(frame, 0.7, road_overlay, 0.3, 0)
            
            return frame, road_mask, fg_mask



    def process_video(self, video_path):
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            print("Error: Could not open video")
            return
        
        frame_count = 0
        while True:
            ret, frame = cap.read()
            if not ret:
                break
                
            processed_frame, road_mask, fg_mask = self.process_frame(frame, frame_count)
            
            cv2.imshow('Road Mask', road_mask)
            cv2.imshow('Foreground Mask', fg_mask)
            cv2.imshow('Tracking', processed_frame)
            
            frame_count += 1
            
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
                
        cap.release()
        cv2.destroyAllWindows()

tracker = VideoProcessor()
tracker.process_video('Images/traffic.mp4')